In [2]:
import pandas as pd
import numpy as np

In [2]:
submissions = ['resnet', 'keras_baseline']

In [5]:
submissions_dfs = []
for submission in submissions:
    submission_df = pd.read_csv("/mnt/extDisk/courses/data/statoil/data/" + submission + "/" + submission + ".csv")
    submission_df = submission_df.sort('id')
    submissions_dfs.append(submission_df)

/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  after removing the cwd from sys.path.


In [72]:
submissions_dfs = []
#paths = ['data_aug/fliping_3_arch.csv', 'ensamble/ensamble_0211.csv','pseudo_labeling/pseu_label_sure_resnet_regular.csv', 'data_aug/black_box_10_learning.csv','data_aug/noise_box_10_learning.csv']
paths = ['data_aug/noise_mix_clean_learning_rate_1.csv','ensamble/stacking_0612.csv']
for path in paths:
    submission_df = pd.read_csv("/mnt/extDisk/courses/data/statoil/data/" + path)
    submission_df = submission_df.sort('id')
    submissions_dfs.append(submission_df)

/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  


In [73]:
ensamble = submissions_dfs[0]
ensamble['is_iceberg'] = np.mean([submission.is_iceberg for submission in submissions_dfs],axis=0)

In [74]:
ensamble.head()

,id,is_iceberg
5442,000c65d7,0.859357
37,001680af,0.148927
697,00193999,0.986860
1152,00247e48,0.951438
3033,00503faa,0.000015


In [75]:
ensamble.to_csv("/mnt/extDisk/courses/data/statoil/data/ensamble/ensamble_0612.csv", index=False)

In [17]:
def mean_distance(df1,df2):
    print np.mean(np.absolute(df1['is_iceberg'] - df2['is_iceberg']))

In [18]:
mean_distance(submissions_dfs[0],submissions_dfs[1])

0.376096297832


In [65]:
for i in range(len(submissions_dfs)):
    submissions_dfs[i] = submissions_dfs[i].rename(columns={'is_iceberg':'is_iceberg_' +str(i)})

concat_sub = submissions_dfs[0]

for i in range(1,len(submissions_dfs)):
    concat_sub = concat_sub.merge(submissions_dfs[i],on='id')

#concat_sub.reset_index(inplace=True)
concat_sub.head()

,id,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3,is_iceberg_4
0,000c65d7,0.807972,0.840560,0.510584,0.692546,0.956177
1,001680af,0.299221,0.209411,0.165921,0.052698,0.705793
2,00193999,0.915564,0.883449,0.984035,0.961919,0.994329
3,00247e48,0.993899,0.990012,0.995281,0.611009,0.796973
4,00503faa,0.000010,0.000414,0.094459,0.009278,0.019158


In [66]:
# get the data fields ready for stacking
concat_sub['is_iceberg_max'] = concat_sub.iloc[:, 1:].max(axis=1)
concat_sub['is_iceberg_min'] = concat_sub.iloc[:, 1:].min(axis=1)
concat_sub['is_iceberg_mean'] = concat_sub.iloc[:, 1:].mean(axis=1)
concat_sub['is_iceberg_median'] = concat_sub.iloc[:, 1:].median(axis=1)
concat_sub.head()

,id,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3,is_iceberg_4,is_iceberg_max,is_iceberg_min,is_iceberg_mean,is_iceberg_median
0,000c65d7,0.807972,0.840560,0.510584,0.692546,0.956177,0.956177,0.510584,0.753514,0.780743
1,001680af,0.299221,0.209411,0.165921,0.052698,0.705793,0.705793,0.052698,0.313076,0.254316
2,00193999,0.915564,0.883449,0.984035,0.961919,0.994329,0.994329,0.883449,0.945296,0.953608
3,00247e48,0.993899,0.990012,0.995281,0.611009,0.796973,0.995281,0.611009,0.856209,0.923111
4,00503faa,0.000010,0.000414,0.094459,0.009278,0.019158,0.094459,0.000010,0.031113,0.014218


In [67]:
# set up cutoff threshold for lower and upper bounds, easy to twist 
cutoff_lo = 0.8
cutoff_hi = 0.2

In [68]:
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_median']))
concat_sub['is_iceberg'].head()

0    0.780743
1    0.254316
2    0.994329
3    0.923111
4    0.000010
Name: is_iceberg, dtype: float64

In [69]:
concat_sub[['id','is_iceberg']].head()

,id,is_iceberg
0,000c65d7,0.780743
1,001680af,0.254316
2,00193999,0.994329
3,00247e48,0.923111
4,00503faa,0.000010


In [71]:
concat_sub[['id','is_iceberg']].to_csv("/mnt/extDisk/courses/data/statoil/data/ensamble/stacking_0612.csv", index=False)

,id,is_iceberg_1,is_iceberg_2,is_iceberg_3,is_iceberg_4,is_iceberg_5,is_iceberg_max,is_iceberg_min,is_iceberg_mean,is_iceberg_median,is_iceberg
0,5941774d,0.032017,0.84056,0.010108,0.069945,0.004346,0.84056,0.004346,0.257412,0.050981,0.050981
